# Extraer Tuits 🐦 


<br><b>Creador:</b> Norma Yuliana Cala Martínez
<br><b>Email: </b> normisss11@gmail.com
<br><b>Fecha creacion:</b> Noviembre 12, 2020

### Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API


# Autenticarse con Twitter

In [6]:
#!pip install tweepy
import pandas as pd 
import tweepy

In [9]:
# Leer keys
keys = pd.read_csv("Twitter_Keys_Text.csv", header=None)
keys = dict(zip(keys[0],keys[1]))

In [11]:
keys

{'api_key': '0RqufMUp83UPzVMiF1vp01n3U',
 'api_secret_key': 'KMStadxEIVBuCLxYeBrsSSWUSJebvwBHABb8RtFCILIOcS5HbL',
 'access_token': '1295837312090353665-9XzYh9ClTf9hwVwc29UQMOj8h37Ovg',
 'access_token_secret': 'Q4M70K4gP6G3xoCpX8gloO8YR3fgQLpdqX2SxLYrf5osW'}

In [12]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [13]:
api.update_status('La mejor clase es NLP.')

TweepError: [{'code': 187, 'message': 'Status is a duplicate.'}]

## Datos de la Red social de Twitter. 

In [14]:
# Tema seleccionado son las eliminatorias al mundial de futbol.  ⚽
tweets = tweepy.Cursor(api.search,
                       q=['eliminatorias -RT'],
                       lang='es',
                       geocode=f'9.304744,-75.397778,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(200)

In [15]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("colombia")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [16]:
t

[['@campoeliasjr Campito El partido de ayer para el olvido, en lo colectivo e individual muy muy malo, no salió nada, para mí el único que vi bien fue a Mojica, el resto ninguno estuvo, pero bueno esperemos este sea el partido malo que nos tocaba en eliminatorias, Vamos @Selecioncolomb1',
  datetime.datetime(2020, 11, 14, 11, 16, 17),
  'colombia',
  'Sincelejo - Sucre',
  1327571047332372480,
  'https://twitter.com/i/web/status/1327571047332372480'],
 ['Una cosa, viendo el calendario que hay en Conmebol, me doy cuenta que no debemos sorprendernos si empatamos o perdemos el siguiente partido...porque es con Brasil, que desde que llegó Tite lleva 0 derrotas en eliminatorias...y solo 2 empates (1 de ellos con nosotros).',
  datetime.datetime(2020, 11, 14, 3, 4, 18),
  'colombia',
  'Sincelejo, Sucre',
  1327447234829447168,
  'https://twitter.com/i/web/status/1327447234829447168'],
 ['Desde que Tite llegó, hay más títulos (1) que derrotas en eliminatorias. https://t.co/fkGFH4f28Z',
  dat

In [17]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,@campoeliasjr Campito El partido de ayer para ...,2020-11-14 11:16:17,colombia,Sincelejo - Sucre,1327571047332372480,https://twitter.com/i/web/status/1327571047332...
1,"Una cosa, viendo el calendario que hay en Conm...",2020-11-14 03:04:18,colombia,"Sincelejo, Sucre",1327447234829447168,https://twitter.com/i/web/status/1327447234829...
2,"Desde que Tite llegó, hay más títulos (1) que ...",2020-11-14 02:58:20,colombia,"Sincelejo, Sucre",1327445734334091264,https://twitter.com/i/web/status/1327445734334...
3,¡ATENCIÓN! #DEPORTES \nBrasil 1-0 Venezuela e...,2020-11-14 02:53:35,colombia,"Córdoba, Colombia",1327444540811259904,https://twitter.com/i/web/status/1327444540811...
4,#ArturoVidal (33) con la selección de Chile🇨🇱:...,2020-11-14 02:41:33,colombia,"Córdoba, Colombia",1327441511361941504,https://twitter.com/i/web/status/1327441511361...


In [18]:
# Exportar datos.
data.to_csv("datos13112020.csv", index=False)

##  Pre-procesamiento para Twitter 
⚽

In [19]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,@campoeliasjr Campito El partido de ayer para ...,2020-11-14 11:16:17,colombia
1,"Una cosa, viendo el calendario que hay en Conm...",2020-11-14 03:04:18,colombia
2,"Desde que Tite llegó, hay más títulos (1) que ...",2020-11-14 02:58:20,colombia
3,¡ATENCIÓN! #DEPORTES \nBrasil 1-0 Venezuela e...,2020-11-14 02:53:35,colombia
4,#ArturoVidal (33) con la selección de Chile🇨🇱:...,2020-11-14 02:41:33,colombia


### Quitar links, menciones, hashtags, Quitar emojis

In [20]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [21]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,@campoeliasjr Campito El partido de ayer para ...,2020-11-14 11:16:17,colombia,"Campito El partido de ayer para el olvido, en...",[],[],"[@campoeliasjr, @Selecioncolomb1]"
1,"Una cosa, viendo el calendario que hay en Conm...",2020-11-14 03:04:18,colombia,"Una cosa, viendo el calendario que hay en Conm...",[],[],[]
2,"Desde que Tite llegó, hay más títulos (1) que ...",2020-11-14 02:58:20,colombia,"Desde que Tite llegó, hay más títulos (1) que ...",[https://t.co/fkGFH4f28Z],[],[]
3,¡ATENCIÓN! #DEPORTES \nBrasil 1-0 Venezuela e...,2020-11-14 02:53:35,colombia,¡ATENCIÓN! \nBrasil 1-0 Venezuela en la fech...,[https://t.co/9cDtASzAVs],"[#DEPORTES, #ChicaNoticias]",[]
4,#ArturoVidal (33) con la selección de Chile🇨🇱:...,2020-11-14 02:41:33,colombia,(33) con la selección de Chile🇨🇱:\n\n•118 Par...,[https://t.co/3FqwrIt0qg],[#ArturoVidal],[]


In [22]:
#!pip install emoji
import regex
import emoji

In [23]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list
data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [24]:
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,@campoeliasjr Campito El partido de ayer para ...,2020-11-14 11:16:17,colombia,"Campito El partido de ayer para el olvido, en...",[],[],"[@campoeliasjr, @Selecioncolomb1]",[]
1,"Una cosa, viendo el calendario que hay en Conm...",2020-11-14 03:04:18,colombia,"Una cosa, viendo el calendario que hay en Conm...",[],[],[],[]
2,"Desde que Tite llegó, hay más títulos (1) que ...",2020-11-14 02:58:20,colombia,"Desde que Tite llegó, hay más títulos (1) que ...",[https://t.co/fkGFH4f28Z],[],[],[]
3,¡ATENCIÓN! #DEPORTES \nBrasil 1-0 Venezuela e...,2020-11-14 02:53:35,colombia,¡ATENCIÓN! \nBrasil 1-0 Venezuela en la fech...,[https://t.co/9cDtASzAVs],"[#DEPORTES, #ChicaNoticias]",[],[]
4,#ArturoVidal (33) con la selección de Chile🇨🇱:...,2020-11-14 02:41:33,colombia,(33) con la selección de Chile🇨🇱:\n\n•118 Par...,[https://t.co/3FqwrIt0qg],[#ArturoVidal],[],"[🇨🇱, 📍, 🤴]"


### Punto 2: Hacer una visualización sobre esos tuits


In [25]:
#!pip install plotly

## Series de tiempo

In [26]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
6,2020-11-05,1
5,2020-11-07,2
3,2020-11-08,5
4,2020-11-09,3
7,2020-11-10,1
1,2020-11-12,10
0,2020-11-13,17
2,2020-11-14,8


In [27]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [28]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Barras

In [29]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
16,🚨,4
5,🎥,2
11,👕,2
9,🤩,2
8,🇨🇴,2


In [30]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [ ]:
import datetime

temp_emoji = []
days = [datetime.date(2020, 11, 7), datetime.date(2020, 11, 6),
       datetime.date(2020, 11, 5), datetime.date(2020, 11, 4),
       datetime.date(2020, 11, 3), datetime.date(2020, 11, 2),
       datetime.date(2020, 11, 1), datetime.date(2020, 10, 31),
       datetime.date(2020, 10, 30), datetime.date(2020, 10, 29)]

for day in days:
    temp = pd.DataFrame(Counter(sum(data[data['fecha']==day]['emojis'].values,[])).most_common(10))
    temp['day'] = day
    temp_emoji.append(temp)
    
temp_emoji = pd.concat(temp_emoji)
temp_emoji.columns = ['emojis', 'tweet_count', 'day']

temp_emoji = temp_emoji.pivot(index='emojis', columns='day').reset_index()
temp_emoji.columns = ['emojis'] + list(temp_emoji.columns.get_level_values(1)[1:])

temp_emoji

In [ ]:
listOfFrames = []
for year in temp_emoji.columns[1:]:
    temp = temp_emoji[['emojis', year]]
    temp = temp.sort_values(year, ascending=False).dropna()
    
    trace = go.Bar(
                x=temp.emojis.values,
                y=temp[year].values,
                textposition = "outside",
                texttemplate = "%{x}<br>%{y}",
                textfont = {'size': [14]*10},
                hoverinfo='none', 
                opacity=1,
                marker=dict(color="orange"),
                cliponaxis = False
                )

    layout = go.Layout(title=f'Emojis del día {year.date()}',
                       xaxis = {"range":[-1,10],"autorange": False, "showline":False,"tickangle":-90, "visible":False},
                       yaxis = {"range":[-1,70],"autorange": False, "showline":False, "visible":False},
                       plot_bgcolor= '#FFFFFF')
    listOfFrames.append(go.Frame(data=[trace], layout=layout))
    
frames=list(listOfFrames)

year = beg
temp = temp_emoji[['emojis', year]]
temp = temp.sort_values(year, ascending=False).dropna()

# Plot
trace = go.Bar(
                x=temp.emojis.values,
                y=temp[year].values,
                textposition = "outside",
                texttemplate = "%{x}<br>%{y}",
                textfont = {'size': [14]*10},
                hoverinfo='none', 
                opacity=1,
                marker=dict(color="orange"),
                cliponaxis = False
                )

layout = go.Layout(title=f'Emojis del día {year.date()}',
                   xaxis = {"range":[-1,10],"autorange": False, "showline":False,"tickangle":-90, "visible":False},
                   yaxis = {"range":[-1,70],"autorange": False, "showline":False, "visible":False},
                   plot_bgcolor= '#FFFFFF',
                   updatemenus=[dict(type="buttons",
                                     buttons=[dict(label="▶️",
                                                   method="animate",
                                                   args=[None, {"frame": {"duration": 800, "redraw": True},
                                                                "fromcurrent": True}]),
                                              {
                "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "⏸️",
                "method": "animate"
            }
                                             ]
                                    )
                               ]
                  )

fig = go.Figure(data=[trace], layout=layout, frames=frames)

fig.show()

In [34]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto

tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado, ngram_range=(1,1), max_features=100)
tfidf = tfidf_vect.fit_transform(data.clean.values)

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]
temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.T.values)
temp = pd.DataFrame(temp)
temp.columns = voc
temp.index = voc
temp

,acá,ahí,alarma,antena,aquí,arrastrándose,así,atención,barranquilla,bien,...,sudamericanas,tercera,tite,todas,transmitir,uruguay,vamos,venezuela,vez,vuelven
acá,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.723345,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
ahí,0.0,1.000000,0.000000,0.000000,0.000000,0.577441,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.454287,0.0
alarma,0.0,0.000000,1.000000,0.644493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
antena,0.0,0.000000,0.644493,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
aquí,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.281258,0.802022,0.000000,0.000000,...,0.357257,0.00000,0.0,0.0,0.0,0.313085,0.0,0.396418,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uruguay,0.0,0.000000,0.000000,0.000000,0.313085,0.000000,0.181949,0.538864,0.432091,0.000000,...,0.271968,0.32688,0.0,0.0,0.0,1.000000,0.0,0.132275,0.000000,0.0
vamos,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081016,...,0.000000,0.00000,0.0,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.0
venezuela,0.0,0.000000,0.000000,0.000000,0.396418,0.000000,0.000000,0.317936,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.0,0.132275,0.0,1.000000,0.000000,0.0
vez,0.0,0.454287,0.000000,0.000000,0.000000,0.786725,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000,0.0


In [36]:
trace = go.Heatmap(z=temp.values.tolist(),
                   x=temp.index.values,
                   y=temp.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre palabras 🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)

In [37]:
import scipy.cluster.hierarchy as sch
import numpy as np

pairwise_distances = sch.distance.pdist(temp)
linkage = sch.linkage(pairwise_distances, method='ward')
idx_to_cluster_array = sch.fcluster(linkage, pairwise_distances.max() * 1, criterion='distance')
idx = np.argsort(idx_to_cluster_array)
temp = temp.copy()
    
temp2 = temp.iloc[idx, :].T.iloc[idx, :]
my_idx = idx_to_cluster_array

In [38]:
trace = go.Heatmap(z=temp2.values.tolist(),
                   x=temp2.index.values,
                   y=temp2.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre palabras (ordenado)🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)